In [1]:
import pandas as pd

file_list = ["AlwaysTraf/1.csv",
             "AlwaysTraf/2.csv",
             "AlwaysTraf/3.csv",
             "AlwaysTraf/4.csv",
             "AlwaysTraf/5.csv",
             "AlwaysTraf/6.csv"]

patterns = ['ARP']
unique_values_protocol = [] # список для хранения протоколов в дампе
lengths_always = [] # список для записи всех уникальных значений максимального объема пакета
end_length_list = [] # список, в котором будет храниться единственное и максимальное значение объема пакета 
#--------------------------------------------------------------------------------------------------
# методы для анализа нормальных дампов
#--------------------------------------------------------------------------------------------------
def first(): # считаем количество пакетов ARP протокола в нормальном трафике
    countARP = 0
    for pattern in patterns:
        filter_result = df[df['Protocol'].str.contains(pattern)]
        count = len(filter_result)
        if pattern == 'ARP':
            countARP += count
    return countARP

def second(): # записываем уникальные протоколы в нормальном трафике
    unique_values = df['Protocol'].unique()
    for value in unique_values:
        unique_values_protocol.append(value)

def get_max_length(df): # получаем максимальное значение объема пакета
    max_length = df['Length'].max()
    return max_length

for i, file in enumerate(file_list, start=1): #цикл для прохода по всем дампам нормального трафика
    df = pd.read_csv(file)
    first()
    second()
    max_len = get_max_length(df)
    lengths_always.append(max_len) # получаем максимальное значение в 1-6 дампе нормального трафика

end_length_list =  list(set(lengths_always)) # удаляем повторяющиеся значения максимального объема

def remove_duplicates(values): # удаляем повторяющиеся значения протоколов 
    unique_values = list(set(values))
    return unique_values

final_list = []
final_list.extend(unique_values_protocol)
unique_values = remove_duplicates(final_list)

print("Данные нормального трафика получены")
print("")

#--------------------------------------------------------------------------------------------------
# выявление аномалий.
# Проверка происходит по одному файлу. Для проверки другого - нужно установить соответствующее
# значение в 120-ой строке, (file_list_anomal[$])
#--------------------------------------------------------------------------------------------------

file_list_anomal = ["AnomalTraf/6.csv",
                    "AnomalTraf/7.csv",
                    "AnomalTraf/8.csv",
                    "AnomalTraf/9.csv"]

real_arp = 0
ddos_count = 0
ddos_porog = 100
prev_no = None
normalArms = first()
normalArms = normalArms - 10
protocol_values_list = []
anomal_pattern = []

DatFr = pd.read_csv(file_list_anomal[0])

def find_length_anomalies(end_length_list, DatFr): # поиск аномалий объема пакета
    for index, row in DatFr.iterrows():
        length = row['Length']
        for max_length in end_length_list:
            if max_length < length:
                protocol = row['Protocol']
                source = row['Source']
                destination = row['Destination']
                info = row['Info']
                print(f"Детектирована аномалия: слишком большой пакет информации. Протокол: {protocol}, IP отправителя: {source}, IP получателя: {destination}, Значение: {info}, Размер: {length}")
                print("")

find_length_anomalies(end_length_list, DatFr)

def search_anomal_protocol(): # ищем протоколы в аномальном трафике
    protocol_values = DatFr['Protocol'].unique()
    for protocol in protocol_values:
        protocol_values_list.append(protocol)

search_anomal_protocol()

def compare_lists(unique_values, protocol_values_list): # сравниваем списки протоколов: нормального и аномального трафика
    for pattern in protocol_values_list:
        if pattern not in unique_values:
            anomal_pattern.append(pattern)
    return anomal_pattern

anomal_pattern = compare_lists(unique_values, protocol_values_list)

compare_lists(unique_values, protocol_values_list)

def find_pattern_values(DatFr, anomal_pattern):
    for index, row in DatFr.iterrows():
        for pattern in anomal_pattern:
            if pattern in row['Protocol']:
                print(f"Детектирована аномалия: {pattern}, IP отправителя: {row['Source']}, IP получателя: {row['Destination']}, Значение: {row['Info']}")
                print("")
                break  

find_pattern_values(DatFr, anomal_pattern)


for index, row in DatFr.iterrows(): # вылавливаем новые подключения и DDoS атаки 
    if row['Protocol'] == 'ARP': 
        real_arp += 1
        if real_arp > normalArms * 2:
            print(f"Детектировано подключение неизвестного устройства.\nARP подключение")
    elif row['Protocol'] == 'TCP': # Отлов Ddos
        # Поиск слова 'SYN' в столбце 'Info'
        if 'SYN' in row['Info']:
            # Запоминаем номер записи
            if prev_no is None:
                prev_no = row['No.']
            else:
                if (row['No.'] - prev_no) == 1:
                    ddos_count += 1
                    if ddos_count >= ddos_porog:
                        print(f"Детектирована DDoS атака\nНачало {index + 1 - ddos_count}")
                        break
                prev_no = row['No.']
        else:
            ddos_count = 0


Данные нормального трафика получены

Детектирована аномалия: слишком большой пакет информации. Протокол: DNS, IP отправителя: 192.168.10.8, IP получателя: 56.110.14.82, Значение: Unknown operation (8) 0x4141[Malformed Packet], Размер: 10042

Детектирована аномалия: IEC 60870-5-104, IP отправителя: 192.168.10.5, IP получателя: 192.168.10.4, Значение: 2404  >  8446 [<None>] Seq=1 Win=596 Len=80

Детектирована аномалия: ZEBRA, IP отправителя: 192.168.10.3, IP получателя: 192.168.10.8, Значение: Zebra Request: Command Type 0x88

Детектирована аномалия: MEMCACHE, IP отправителя: 192.168.10.2, IP получателя: 192.168.10.6, Значение: MEMCACHE Continuation

Детектирована аномалия: HTTP, IP отправителя: 192.168.10.5, IP получателя: 216.58.209.238, Значение: GET / HTTP/1.1 

Детектирована аномалия: HTTP, IP отправителя: 192.168.10.5, IP получателя: 216.58.209.238, Значение: GET / HTTP/1.1 

Детектирована аномалия: Gearman, IP отправителя: 192.168.10.10, IP получателя: 192.168.10.4, Значение: [MGR